In [1]:
import yaml
from rosbag.bag import Bag

In [2]:
bag1_filename = 'rosbags/record_2024_03_27_10_51_34.bag'
bag2_filename = 'rosbags/record_2024_03_27_11_03_09.bag'

In [3]:
bag = Bag(bag2_filename, 'r')
bag_info = yaml.load(bag._get_yaml_info(), Loader=yaml.Loader)
with open('bag_info.yaml', 'w') as file:
    yaml.dump(bag_info, file)

In [4]:
topics = bag.get_type_and_topic_info()[1].keys()
types = [val[0] for val in bag.get_type_and_topic_info()[1].values()]

print('Topics:', topics)
print('Types:', types)

Topics: dict_keys(['/audio', '/joint_states', '/xtion/rgb/image_raw_throttled'])
Types: ['audio_common_msgs/AudioData', 'sensor_msgs/JointState', 'sensor_msgs/Image']


In [5]:
import cv2
from cv_bridge import CvBridge

bridge = CvBridge()

for i, (_, message, t) in enumerate(bag.read_messages(topics='/xtion/rgb/image_raw_throttled')):
    cv2_image = bridge.imgmsg_to_cv2(message, desired_encoding='passthrough')
    cv2_image = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(f'images/frame{i:04d}.jpg', cv2_image)

In [6]:
# Extract video
def get_fps():
    for topic in bag_info['topics']:
        if topic['type'] == 'sensor_msgs/Image':
            return topic['frequency']
    return None

import subprocess
subprocess.run(['ffmpeg', '-r', str(get_fps()), '-i', r'images/frame%04d.jpg', 'video.avi'])

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass -

CompletedProcess(args=['ffmpeg', '-r', '3.7524', '-i', 'images/frame%04d.jpg', 'video.avi'], returncode=0)

In [7]:
# Extract audio
with open('audio.mp3', 'wb') as f:
    for i, (_, message, t) in enumerate(bag.read_messages(topics='/audio')):
        for byte in message.data:
            f.write(int.to_bytes(byte))

In [8]:
# Find audio delay
first_video_timestamp_ns = next(bag.read_messages(topics='/xtion/rgb/image_raw_throttled')).timestamp
first_audio_timestamp_ns = next(bag.read_messages(topics='/audio')).timestamp
audio_delay_in_seconds = (first_audio_timestamp_ns - first_video_timestamp_ns) / 1000000000
print(audio_delay_in_seconds)

0


In [9]:
# Combine video and audio
subprocess.run(['ffmpeg', '-i', 'video.avi', '-i', 'audio.mp3', '-c:v', 'copy', '-map', '0:v', '-map', '1:a', '-y', 'combined.mp4'])

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass -

CompletedProcess(args=['ffmpeg', '-i', 'video.avi', '-i', 'audio.mp3', '-c:v', 'copy', '-map', '0:v', '-map', '1:a', '-y', 'combined.mp4'], returncode=0)

In [10]:
subprocess.run(['ffmpeg', '-i', 'combined.mp4', '-vcodec', 'libx264', '-acodec', 'copy', 'output.mp4'])

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 16.0.6
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1710227269443/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass -

CompletedProcess(args=['ffmpeg', '-i', 'combined.mp4', '-vcodec', 'libx264', '-acodec', 'copy', 'output.mp4'], returncode=0)

In [1]:
import os
os.remove('bag_info.yaml')
os.remove('combined.mp4')
os.remove('video.avi')